In [1]:
import json
import pandas as pd
import os
import shutil
import datetime

In [2]:
DATA_DIR = "../data"
DATA_FILE = os.path.join(DATA_DIR, "data.json")
DUMP_FILE = os.path.join(DATA_DIR, "dump.pkl")
CURRENT_YEAR = int(datetime.datetime.now().date().strftime("%Y"))

In [3]:
store_columns = (
    "id",  # 음식점 고유번호
    "store_name",  # 음식점 이름
    "branch",  # 음식점 지점 여부
    "area",  # 음식점 위치
    "tel",  # 음식점 번호
    "address",  # 음식점 주소
    "latitude",  # 음식점 위도
    "longitude",  # 음식점 경도
    "category",  # 음식점 카테고리
    "review_cnt" # 평가 수
)

review_columns = (
    "id",  # 리뷰 고유번호
    "store",  # 음식점 고유번호
    "user",  # 유저 고유번호
    "score",  # 평점
    "content",  # 리뷰 내용
    "reg_time",  # 리뷰 등록 시간
)

menu_columns = (
    "id",
    "store",
    "menu_name",
    "price"
)

user_columns = (
    "id",
    "gender",
    "age"
)

In [4]:
from parse import load_dataframes
import pandas as pd
import shutil

In [47]:
def import_data(data_path=DATA_FILE):
    """
    Req. 1-1-1 음식점 데이터 파일을 읽어서 Pandas DataFrame 형태로 저장합니다
    """
    try:
        with open(data_path, encoding="utf-8") as f:
            data = json.loads(f.read())
    except FileNotFoundError as e:
        print(f"`{data_path}` 가 존재하지 않습니다.")
        exit(1)

    stores = []  # 음식점 테이블
    reviews = []  # 리뷰 테이블
    menus = [] # 메뉴 테이블
    users = [] # 유저 테이블
    user_duplicate = set()
    menu_cnt = 0

    for d in data:
        categories = [c["category"] for c in d["category_list"]]
        stores.append(
            [
                d["id"],
                d["name"],
                d["branch"],
                d["area"],
                d["tel"],
                d["address"],
                d["latitude"],
                d["longitude"],
                "|".join(categories),
                d["review_cnt"]
            ]
        )

        for review in d["review_list"]:
            r = review["review_info"]
            u = review["writer_info"]

            reviews.append(
                [r["id"], d["id"], u["id"], r["score"], r["content"], r["reg_time"]]
            )
            # Req. 1-1 / user, 한국식 나이
            if u["id"] not in user_duplicate:
                users.append(
                    [u["id"], u["gender"], CURRENT_YEAR - int(u["born_year"]) + 1]
                )
                user_duplicate.add(u["id"])

        # Req. 1-1 / menu
        for m in d["menu_list"]:
            menus.append(
                [menu_cnt, d["id"], m["menu"], m["price"]]
            )
            menu_cnt += 1

    store_frame = pd.DataFrame(data=stores, columns=store_columns)
    review_frame = pd.DataFrame(data=reviews, columns=review_columns)
    menu_frame = pd.DataFrame(data=menus, columns=menu_columns)
    user_frame = pd.DataFrame(data=users, columns=user_columns)

    return {"stores": store_frame, "reviews": review_frame, "menus": menu_frame, "users": user_frame}


def dump_dataframes(dataframes):
    pd.to_pickle(dataframes, DUMP_FILE)


def load_dataframes():
    return pd.read_pickle(DUMP_FILE)

In [48]:
dataframes = load_dataframes()
stores_reviews = pd.merge(
    dataframes["stores"], dataframes["reviews"], left_on="id", right_on="store"
)
scores_reviews_group = stores_reviews[stores_reviews['review_cnt'] >= 10].groupby(["store", "store_name", "branch", "review_cnt"])
# 리뷰 개수 필터링
# scores_reviews_group.filter(lambda x: x['review_cnt'].count() >= 30)
scores_mean = scores_reviews_group.mean() # 평균 평점 구하기

# Req 1-2-1
stores_sorted_by_scores = scores_mean.sort_values(by='score', ascending=False)

In [7]:
scores_reviews_group

In [8]:
scores_mean

,,,,id_x,id_y,user,score
store,store_name,branch,review_cnt,,,,
221,10Q,,12,221.0,82.5,153224.083333,5.000000
13301,가미우동,서교점,12,13301.0,4517.5,148291.166667,4.416667
17251,감칠,,23,17251.0,5564.0,386409.956522,3.869565
17956,강강술래,상계점,10,17956.0,5770.5,365367.400000,3.400000
18359,강남불백,1호점,13,18359.0,5926.0,441764.076923,4.153846
...,...,...,...,...,...,...,...
346836,정돈,,65,346836.0,11646.0,285306.923077,4.430769
350331,제주김만복,본점,19,350331.0,162.0,263377.578947,3.526316
352194,조대포,남영본점,13,352194.0,770.0,308155.000000,3.692308


In [9]:
stores_sorted_by_scores

,,,,id_x,id_y,user,score
store,store_name,branch,review_cnt,,,,
221,10Q,,12,221.0,82.5,153224.083333,5.000000
149760,모연,,13,149760.0,15789.0,226549.923077,5.000000
249786,신림 춘천집,,11,249786.0,18736.0,201227.000000,5.000000
118037,라라브레드,,14,118037.0,3856.5,213702.357143,4.928571
129842,마이란,,11,129842.0,10424.0,204051.363636,4.909091
...,...,...,...,...,...,...,...
241641,스시메이진,홍대점,10,241641.0,13394.5,369596.000000,3.000000
133541,만석닭강정,본점,11,133541.0,11407.0,560045.727273,2.909091
119000,라운드어바웃,한남점,10,119000.0,6783.5,376401.600000,2.900000


In [10]:
stores_reviewss = pd.merge(
    dataframes["stores"], dataframes["reviews"], left_on="id", right_on="store"
)
reviews = stores_reviews.groupby(["store", "store_name", "branch", "review_cnt", "tel", "area"])
reviews_mean = reviews.mean("review_cnt")

stores_sorted_by_reviews = reviews_mean.sort_values(by='review_cnt', ascending=False)

In [11]:
reviews

In [12]:
reviews_mean

,,,,,,id_x,id_y,user,score
store,store_name,branch,review_cnt,tel,area,,,,
15,써리힐,,1,010-7574-4533,전포동,15.0,1.0,68632.0,5.0
18,진삼미 샌드위치,,1,063-221-9801,전주,18.0,2.0,389728.0,5.0
19,한옥마을 전주비빔밥,향남점,1,,향남지구,19.0,3.0,68716.0,4.0
53,0430 국물닭갈비,,1,055-248-0430,마산,53.0,9.0,52424.0,5.0
91,10 PIPE,,1,031-962-2110,삼송,91.0,26.0,168720.0,5.0
...,...,...,...,...,...,...,...,...,...
360070,쥬씨,인천용현동점,1,070-4575-8254,인천용현동,360070.0,2311.0,361309.0,3.0
360071,쥬씨,석촌역점,1,0507-1413-2802,석촌역,360071.0,2312.0,391153.0,5.0
360153,쥬씨,독산역점,2,02-830-1112,독산역,360153.0,2316.5,143764.5,4.5


In [13]:
stores_sorted_by_reviews

,,,,,,id_x,id_y,user,score
store,store_name,branch,review_cnt,tel,area,,,,
145030,명동교자,본점,101,02-776-5348,명동,145030.0,14329.0,354779.099010,4.158416
346836,정돈,,65,02-987-0924,대학로,346836.0,11646.0,285306.923077,4.430769
305906,우래옥,본점,63,02-2265-0151,을지로,305906.0,1280.0,365475.428571,4.095238
106124,동화가든,본점,59,033-652-9885,강릉,106124.0,1307.0,319598.050847,4.067797
321390,을밀대,본점,49,02-717-1922,마포,321390.0,5458.0,380157.591837,4.285714
...,...,...,...,...,...,...,...,...,...
146336,명륜진사갈비,서울망우점,1,02-435-4035,망우동,146336.0,14799.0,127124.000000,5.000000
146335,명륜진사갈비,,1,02-935-0556,노원,146335.0,14798.0,391153.000000,5.000000
146332,명륜진사갈비,인천삼산점,1,032-511-2237,굴포천역,146332.0,14794.0,117432.000000,5.000000


In [14]:
stores_reviewss.columns

Index(['id_x', 'store_name', 'branch', 'area', 'tel', 'address', 'latitude',
       'longitude', 'category', 'review_cnt', 'id_y', 'store', 'user', 'score',
       'content', 'reg_time'],
      dtype='object')

In [32]:
def get_most_reviewed_stores(dataframes, n=20):
    """
    Req. 1-2-3 가장 많은 리뷰를 받은 `n`개의 음식점을 정렬하여 리턴합니다
    """
    stores_reviews = pd.merge(
        dataframes["stores"], dataframes["reviews"], left_on="id", right_on="store"
    )
    reviews = stores_reviews.groupby(["store", "store_name", "branch", "review_cnt"])
    reviews_mean = reviews.mean()

    stores_sorted_by_reviews = reviews_mean.sort_values(by='review_cnt', ascending=False)
    return stores_sorted_by_reviews.head(n=n).reset_index()

In [69]:
def get_most_active_users(dataframes, n=20):
    """
    Req. 1-2-4 가장 많은 리뷰를 작성한 `n`명의 유저를 정렬하여 리턴합니다.
    """
    # user와 review를 병합
    users_reviews = pd.merge(
        dataframes["users"], dataframes["reviews"], left_on="id", right_on="user"
    )
    # 확인하고 싶은 컬럼을 넣고, size() 함수를 사용하면 몇 개의 데이터가 group으로 묶였는지 새로운 컬럼이 생성된다. dataFrameGroup의 경우, count()와 같다.
    # 이후 reset_index, rename을 사용해 원하는 컬럼명을 설정한 뒤 사용한다.
    users = users_reviews.groupby(["user", "gender", "age"]).user.size().reset_index(name='reviews_cnt')

    users_sorted_by_review_counts = users.sort_values(by='reviews_cnt', ascending=False).reset_index()
    return users_sorted_by_review_counts

In [70]:
best_reviewers = get_most_active_users(dataframes)

In [71]:
best_reviewers

,index,user,gender,age,reviews_cnt
0,13158,469245,남,37,461
1,8101,243883,남,45,389
2,10471,328775,남,44,380
3,12001,391794,여,31,362
4,2760,74999,여,30,352
...,...,...,...,...,...
18987,7810,224528,남,28,1
18988,7811,224532,남,25,1
18989,7814,224565,남,28,1
18990,7815,224612,여,40,1
